Finder program

In [ ]:
import os
import re
import time
import argparse
import pickle
from threading import Thread

t1 = time.time()
dict1 = {}
i = 1
def get_drives():
	resp= os.popen('wmic logicaldisk get caption')
	drive=resp.read()
	return drive.split()[1:]
	
def creating_index(path):
	global i
	resp=os.walk(path)
	for root,d,files in resp:
		for file in files:
			path1 = root+"\\"+file
			#print(path1)
			file1 = file
			if file1 in dict1:
				file1 = file+"|"+str(i)
				i = i+1
			dict1[file1] = path1
		
def create_index():
	list1_th = []
	for d in get_drives():
		print(d)
		th1 = Thread(target=creating_index, args=(d+"\\",))
		list1_th.append(th1)
		th1.start()
		
	for th1 in list1_th:
		th1.join()
	fw = open("finder.index","wb")
	pickle.dump(dict1,fw)
	fw.close()
	t2 = time.time()
	print("Index created, time taken", t2-t1)
	

def search(file1):
	t3 = time.time()
	fr = open("finder.index","rb")
	data1 = pickle.load(fr)
	fr.close()
	for k,v in dict1.items():
		#print (k,v)
		k = k.split("|")[0]
		m = re.search(file1,k,re.I)
		if m:
			print(k, " : ", v)
	t4 = time.time()
	print("Time taken to search", t4-t3)
		
def main():
	parser = argparse.ArgumentParser()
	parser.add_argument("file_name", nargs="?")
	parser.add_argument("-c", help="Create index", action='store_true')
	args = parser.parse_args()
	print (args)
	try:
		if args.c:
			create_index()
			
		else:
			if args.file_name=="" or args.file_name==None:
				print("Please give the file name")
			else:
				search(args.file_name)
				
	except Exception as e :
		print (e)
		
main()
	



